In [10]:
from unipressed import UniprotkbClient
import itertools as it
from Bio import SeqIO
from tqdm import tqdm
import re
from io import StringIO
import taxoniq

# Actinopterygii

Collect uniprot IDs from Actinopterygii muscle sequences obtained from UniParc.
We only keep those from UniProt.

In [5]:
orthrus_dbs = '../orthrus_db/'
actinopterygii = orthrus_dbs + 'Actinopterygii_muscle.fasta'

In [4]:
actinopterygii_up_ids = []
for r in SeqIO.parse(actinopterygii, "fasta"):
    header = r.id
    parts = header.split('|')
    if len(parts) == 5:
        up_id = parts[3]
        # Remove .version from id
        up_id = up_id.split('.')[0]
        actinopterygii_up_ids.append(up_id)

# Uniprot and SwissProt
Here we collect the unique sequences from:
- SwissProt
- UniprotKB at protein evidence level
- UniprotKB at transcript evidence level
- Any anchovy sequence

In [12]:
swissprot = orthrus_dbs + 'uniprot_sprot.fasta'
swissprot_records = SeqIO.to_dict(SeqIO.parse(swissprot, 'fasta'))
anchovy = orthrus_dbs + 'uniprot_anchovy.fasta'


In [27]:
unique_accs = {}
for r in tqdm(SeqIO.parse(swissprot, 'fasta'), 'Parsing SwissProt'):
    accession = r.id.split('|')[1]
    unique_accs.update({accession: r})
print(len(unique_accs))

Parsing SwissProt: 572970it [00:07, 75635.49it/s] 
Parsin prot evidence: 401433it [00:05, 78689.73it/s] 
Parsing transcript evidence: 1419182it [00:13, 108879.20it/s]

2221253


In [28]:
for r in tqdm(SeqIO.parse(anchovy, 'fasta'), 'Parsing anchovy evidence'):
    accession = r.id.split('|')[1]
    unique_accs.update({accession: r})

Parsing anchovy evidence: 498it [00:00, 76134.99it/s]


In [65]:
SeqIO.write(list(unique_accs.values()), orthrus_dbs + 'uniprot_joint_db.fasta', 'fasta')

2221709

# Cross Actinopterygii and Uniprot
Here we check which Actinopterygii ids are not yet in `unique_accs`.
We collect them and use UniPressed to obtain the Uniprot fastas

In [8]:
# We need to break the IDs into chunks to call UniProt REST API
def chunks(long_list, size):
    long_list = iter(long_list)
    return iter(lambda: tuple(it.islice(long_list, size)), ())

In [9]:
fasta_string = ''
for chunk in tqdm(chunks(actinopterygii_up_ids, 100)):
    for r in UniprotkbClient.fetch_many(chunk, 'fasta'):
        fasta_string += r.decode("utf-8")


105it [01:42,  1.03it/s]


In [10]:
with open(orthrus_dbs + 'Actinopterygii_muscle_uniprot.fasta', 'w') as f:
    f.write(fasta_string)

In [13]:
all_actino = {}
for r in SeqIO.parse(orthrus_dbs + 'Actinopterygii_muscle_uniprot.fasta', 'fasta'):
    all_actino[r.id] = r

anchovy_records = SeqIO.to_dict(SeqIO.parse(anchovy, 'fasta'))
swissprot_records = SeqIO.to_dict(SeqIO.parse(swissprot, 'fasta'))
all_actino.update(anchovy_records)
all_actino.update(swissprot_records)
SeqIO.write(
    list(all_actino.values()),
    orthrus_dbs + 'fish_swissprot_database.fasta',
    'fasta'
)


582701

# Part 2 - Clean orthrus db from non-fish sequences

In [39]:
# Part 2 - Clean Orthrus database from non-fish sequences
orthrus_fasta = orthrus_dbs + 'combined_pt1.fasta'

orthrus_fasta = SeqIO.parse(orthrus_fasta, 'fasta')

ox_re = re.compile('OX=(\d+?) ')

In [40]:
fish_orthrus_database = []
for r in orthrus_fasta:
    m = ox_re.search(r.description)
    taxid = m.group(1)
    try:
        t = taxoniq.Taxon(taxid)
        for l in t.ranked_lineage:
            if l.rank.name == 'class' and l.scientific_name == 'Actinopteri':
                fish_orthrus_database.append(r)
    except KeyError:
        print('.....')
        print(f'Taxid {taxid} not found')
        fish_orthrus_database.append(r)
        print('.....')



.....
Taxid 3356398 not found
.....
.....
Taxid 3362959 not found
.....
.....
Taxid 3357921 not found
.....


In [41]:
SeqIO.write(fish_orthrus_database, orthrus_dbs + 'fish_combined_pt1.fasta', 'fasta')

3713